In [1]:
import pandas as pd
import metadata

In [11]:
# metadata.CAMPOS

In [3]:
df_cnpj = pd.read_csv('data/Estabelecimentos0.zip', 
                    compression='zip', 
                    sep=';', 
                    encoding_errors='replace', 
                    header=None, 
                    names=list(metadata.CAMPOS.keys()),
                    dtype=metadata.CAMPOS,
                    parse_dates=['data_situacao_cadastral'])


In [10]:
df_cnpj.data_situacao_cadastral.str.len().value_counts()


8    13002771
1       29505
Name: data_situacao_cadastral, dtype: int64